In [1]:
import pandas as pd
import numpy as np
import os

### Import the whole dataset

In [2]:
# Dataset load
cars = pd.read_csv("used_car_mini.csv")

# Dataset overview
cars.head(2)

,Unnamed: 0,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


### First create depreciation_info, pickup_truck table

In [3]:

# cars_mini = cars.head(10000)
# Dp = cars_mini[['frame_damaged', 'has_accidents', 'salvage', 'isCab', 'theft_title']]

Dp = cars[['frame_damaged', 'has_accidents', 'salvage', 'isCab', 'theft_title']]
# df_new = df.drop(df[ <condition1> & <condition2> ].index)
Dp = Dp.drop(Dp[(Dp['frame_damaged'] == False) & 
           (Dp['has_accidents'] == False) & 
           (Dp['salvage'] == False) & 
           (Dp['isCab'] == False) & 
           (Dp['theft_title'] == False)].index)
Dp = Dp.dropna(how='all')
Dp.head(2)
# Dp.shape

,frame_damaged,has_accidents,salvage,isCab,theft_title
38,False,False,False,True,False
45,False,True,False,False,False


In [4]:
Dp.shape

(2146, 5)

In [5]:
# Create pickup_truck table
pk = cars[['bed', 'bed_length', 'cabin']]

# Extract bed_length value
temp = pk['bed_length']
temp = temp.replace(r' in', '', regex=True)
temp = temp.replace(r'--', np.nan, regex=True)
temp = temp.astype('float')
pk.loc[:, 'bed_length'] = temp
pk = pk.dropna(how='all')
pk.shape

/Users/lemingqiu/mambaforge/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


(698, 3)

### Then car table

In [6]:
# Create Car df
Car = cars[["vin", "sp_id", "dealer_zip", "city", # seller info attributes
            "price", "year", "make_name", "model_name", "body_type", "maximum_seating", "listing_color", "interior_color", "exterior_color", "mileage",  # car main attributes
            "length", "width", "height", "wheelbase", "front_legroom", "back_legroom",  # car detailed attributes
            "engine_displacement", "engine_type", "transmission", "transmission_display", "wheel_system", "wheel_system_display", "horsepower", "power", "torque",  # power info
            "fuel_tank_volume", "fuel_type", "city_fuel_economy", "highway_fuel_economy",  # economic info
            # 'is_depreciated', 'seller_id', 'country', 'zip', 'power_rpm', 'pound_foot', 'torque_rpm'\
            "is_new", "listed_date", "main_picture_url", 
            "owner_count", "seller_rating", 
            "trim_name",
            'frame_damaged', 'has_accidents', 'salvage', 'isCab', 'theft_title'
            ]]
Car.head(2)

,vin,sp_id,dealer_zip,city,price,year,make_name,model_name,body_type,maximum_seating,...,listed_date,main_picture_url,owner_count,seller_rating,trim_name,frame_damaged,has_accidents,salvage,isCab,theft_title
0,ZACNJABB5KPJ92081,370599.0,960,Bayamon,23141.0,2019,Jeep,Renegade,SUV / Crossover,5 seats,...,2019-04-06,https://static.cargurus.com/images/forsale/202...,NaN,2.8,Latitude FWD,NaN,NaN,NaN,NaN,NaN
1,SALCJ2FX1LH858117,389227.0,922,San Juan,46500.0,2020,Land Rover,Discovery Sport,SUV / Crossover,7 seats,...,2020-02-15,https://static.cargurus.com/images/forsale/202...,NaN,3.0,S AWD,NaN,NaN,NaN,NaN,NaN


In [7]:
# rename sp_id, dealer_zip, etc.
Car = Car.rename(columns={'sp_id': 'seller_id', 'dealer_zip': 'zip', 'power': 'power_rpm', 'torque': 'torque_rpm'})

# insert is_depreciated column and assign values according to damage etc.
Car.insert(1, 'is_depreciated', False)
Car.loc[Car['frame_damaged'] == True, 'is_depreciated'] = True
Car.loc[Car['has_accidents'] == True, 'is_depreciated'] = True
Car.loc[Car['salvage'] == True, 'is_depreciated'] = True
Car.loc[Car['isCab'] == True, 'is_depreciated'] = True
Car.loc[Car['theft_title'] == True, 'is_depreciated'] = True
Car = Car.drop(['frame_damaged', 'has_accidents', 'salvage', 'isCab', 'theft_title'], axis=1)

# insert country column
Car.insert(5, 'country', 'USA')
Car.head(2)

,vin,is_depreciated,seller_id,zip,city,country,price,year,make_name,model_name,...,fuel_tank_volume,fuel_type,city_fuel_economy,highway_fuel_economy,is_new,listed_date,main_picture_url,owner_count,seller_rating,trim_name
0,ZACNJABB5KPJ92081,False,370599.0,960,Bayamon,USA,23141.0,2019,Jeep,Renegade,...,12.7 gal,Gasoline,NaN,NaN,True,2019-04-06,https://static.cargurus.com/images/forsale/202...,NaN,2.8,Latitude FWD
1,SALCJ2FX1LH858117,False,389227.0,922,San Juan,USA,46500.0,2020,Land Rover,Discovery Sport,...,17.7 gal,Gasoline,NaN,NaN,True,2020-02-15,https://static.cargurus.com/images/forsale/202...,NaN,3.0,S AWD


#### Adjust power and torque columns

In [8]:
# Extract power_rpm
temp = Car['power_rpm'].replace(r'.*hp', '', regex=True)
temp = temp.replace(r'.*@ ', '', regex=True)
temp = temp.replace(r' RPM', '', regex=True)
temp = temp.replace(r',', '', regex=True)
temp = temp.replace(r'', np.nan, regex=True)
temp = temp.astype('float')
Car.loc[:, 'power_rpm'] = temp

# insert column pound_foot referring torque and extract torque value(lb-ft)
Car.insert(30, 'pound_foot', Car['torque_rpm'])
temp = Car['pound_foot']
temp = temp.replace(r' lb.*', '', regex=True)
temp = temp.astype('float')
Car.loc[:, 'pound_foot'] = temp

# Extract torque_rpm
temp = Car['torque_rpm']
temp = temp.replace(r'.*lb-ft', '', regex=True)
temp = temp.replace(r'.*@ ', '', regex=True)
temp = temp.replace(r' RPM', '', regex=True)
temp = temp.replace(r',', '', regex=True)
temp = temp.replace(r'', np.nan, regex=True)
temp.astype('float')
Car.loc[:, 'torque_rpm'] = temp


In [9]:
Car.head(2)

,vin,is_depreciated,seller_id,zip,city,country,price,year,make_name,model_name,...,fuel_tank_volume,fuel_type,city_fuel_economy,highway_fuel_economy,is_new,listed_date,main_picture_url,owner_count,seller_rating,trim_name
0,ZACNJABB5KPJ92081,False,370599.0,960,Bayamon,USA,23141.0,2019,Jeep,Renegade,...,12.7 gal,Gasoline,NaN,NaN,True,2019-04-06,https://static.cargurus.com/images/forsale/202...,NaN,2.8,Latitude FWD
1,SALCJ2FX1LH858117,False,389227.0,922,San Juan,USA,46500.0,2020,Land Rover,Discovery Sport,...,17.7 gal,Gasoline,NaN,NaN,True,2020-02-15,https://static.cargurus.com/images/forsale/202...,NaN,3.0,S AWD


In [10]:
Car.head(2)

,vin,is_depreciated,seller_id,zip,city,country,price,year,make_name,model_name,...,fuel_tank_volume,fuel_type,city_fuel_economy,highway_fuel_economy,is_new,listed_date,main_picture_url,owner_count,seller_rating,trim_name
0,ZACNJABB5KPJ92081,False,370599.0,960,Bayamon,USA,23141.0,2019,Jeep,Renegade,...,12.7 gal,Gasoline,NaN,NaN,True,2019-04-06,https://static.cargurus.com/images/forsale/202...,NaN,2.8,Latitude FWD
1,SALCJ2FX1LH858117,False,389227.0,922,San Juan,USA,46500.0,2020,Land Rover,Discovery Sport,...,17.7 gal,Gasoline,NaN,NaN,True,2020-02-15,https://static.cargurus.com/images/forsale/202...,NaN,3.0,S AWD


#### extract values in columns maximum_seating, length, width, height, wheelbase, front_legroom, back_legroom, fuel_tank_volumn

In [11]:
# maximum_seating
temp = Car['maximum_seating']
temp = temp.replace(r' seats', '', regex=True)
temp = temp.replace(r'--', np.nan, regex=True)
temp = temp.astype('float')
Car.loc[:, 'maximum_seating'] = temp

# height length width
temp = Car['height']
temp = temp.replace(r' in', '', regex=True)
temp = temp.replace(r'--', np.nan, regex=True)
temp = temp.astype('float')
temp1 = Car['length']
temp1 = temp1.replace(r' in', '', regex=True)
temp1 = temp1.replace(r'--', np.nan, regex=True)
temp1 = temp1.astype('float')
temp2 = Car['width']
temp2 = temp2.replace(r' in', '', regex=True)
temp2 = temp2.replace(r'--', np.nan, regex=True)
temp2 = temp2.astype('float')
Car.loc[:, 'height'] = temp
Car.loc[:, 'length'] = temp1
Car.loc[:, 'width'] = temp2

# wheelbase
temp = Car['wheelbase']
temp = temp.replace(r' in', '', regex=True)
temp = temp.replace(r'--', np.nan, regex=True)
temp = temp.astype('float')
Car.loc[:, 'wheelbase'] = temp

# front_legroom, back_legroom
temp = Car['front_legroom']
temp = temp.replace(r' in', '', regex=True)
temp = temp.replace(r'--', np.nan, regex=True)
temp = temp.astype('float')
Car.loc[:, 'front_legroom'] = temp
temp = Car['back_legroom']
temp = temp.replace(r' in', '', regex=True)
temp = temp.replace(r'--', np.nan, regex=True)
temp = temp.astype('float')
Car.loc[:, 'back_legroom'] = temp

# fuel_tank_volume
temp = Car['fuel_tank_volume']
temp = temp.replace(r' gal', '', regex=True)
temp = temp.replace(r'--', np.nan, regex=True)
temp = temp.astype('float')
Car.loc[:, 'fuel_tank_volume'] = temp

Car.head(2)

,vin,is_depreciated,seller_id,zip,city,country,price,year,make_name,model_name,...,fuel_tank_volume,fuel_type,city_fuel_economy,highway_fuel_economy,is_new,listed_date,main_picture_url,owner_count,seller_rating,trim_name
0,ZACNJABB5KPJ92081,False,370599.0,960,Bayamon,USA,23141.0,2019,Jeep,Renegade,...,12.7,Gasoline,NaN,NaN,True,2019-04-06,https://static.cargurus.com/images/forsale/202...,NaN,2.8,Latitude FWD
1,SALCJ2FX1LH858117,False,389227.0,922,San Juan,USA,46500.0,2020,Land Rover,Discovery Sport,...,17.7,Gasoline,NaN,NaN,True,2020-02-15,https://static.cargurus.com/images/forsale/202...,NaN,3.0,S AWD


### Next is user table

In [12]:
# Create user table
user = cars[['sp_id', 'sp_name', 'franchise_dealer']]

# Rename columns and insert email and type column
user = user.rename(columns={'sp_id': 'id', 'sp_name': 'name', 'franchise_dealer': 'is_franchise_dealer'})
user.insert(2, 'email', 'xxx@xxx.com')
user.insert(3, 'type', 'dealer')

user.loc[user['is_franchise_dealer'] != True, 'type'] = 'personal'
user = user.drop_duplicates()
# user.head(2)
user.shape

(98, 5)

### export the dataframes to csv files

In [13]:
cwd = os.getcwd()
print (cwd)
Car.to_csv(cwd + '/car.csv')
Dp.to_csv(cwd + '/depreciation_info.csv')
user.to_csv(cwd + '/user.csv', index=False)
pk.to_csv(cwd + '/pickup_truck.csv')
# Car.to_csv(debug_path + 'car.csv')
# Dp.to_csv(debug_path + 'depreciation_info.csv')
# user.to_csv(debug_path + 'user.csv', index=False)
# pk.to_csv(debug_path + 'pickup_truck.csv')

/Users/lemingqiu/Desktop/ECE651/Project/submit/gearmax-data/test_data/mini_data


### Create major_option and car_option table

In [14]:
# Create original major_option table
df_mjr = cars[['major_options']]
df_mjr = df_mjr.drop_duplicates()

# Extract options and create destination table
temp = df_mjr['major_options']
lst_opt = []
for sublist in temp:
    if type(sublist) == str:
        temp_opts = sublist.split(',')
        for item in temp_opts:
            lst_opt.append(item)
lst_opt = list(set(lst_opt))
# print(len(lst_opt), lst_opt)
major_option = pd.DataFrame(lst_opt, columns=['option'])

# strip leading and ending characters
temp = major_option['option']
temp = temp.str.strip('\[\]\' ')
major_option = temp
major_option = major_option.drop_duplicates().reset_index(drop=True)
# major_option = major_option.reindex()

# Create index for major_option table
# major_option = major_option.reset_index()
# major_option = major_option.rename(columns={'index': 'option_id'})
# major_option = major_option[['option_id', 'option']]
# major_option.head(150)

In [15]:
# Get original options for each car
# df_car_opt = cars[['vin', 'major_options']].reset_index()
# df_car_opt = df_car_opt.rename(columns={'index':'car_id'})

# Create data containing pairs of (option_id : car_id)
temp = cars['major_options']
lst_option_id = []
lst_car_id = []
for i in major_option.index:
    # get cars has current option
    curr_option = major_option[i]
    lst_curr_car_id = temp.loc[temp.str.contains(curr_option, na=False)].index.tolist()
    curr_car_num = len(lst_curr_car_id)
    # update pair lists
    lst_option_id += [i] * curr_car_num
    lst_car_id += lst_curr_car_id


In [16]:
# Create car_option table
# lst_option_id = [i for i in range(0, len(major_option)) for j in range(0, len(lst_car_opt[i]))]
# lst_car_id = [lst_car_opt[i][j] for i in range(0, len(major_option)) for j in range(0, len(lst_car_opt[i]))]
car_option = pd.DataFrame(list(zip(lst_option_id, lst_car_id)), columns=['option_id', 'car_id'])

# Export major_option and car_option tables to csv
cwd = os.getcwd()
print (cwd)
major_option.to_csv(cwd + '/major_option.csv')
car_option.to_csv(cwd + '/car_option.csv', index=False)

/Users/lemingqiu/Desktop/ECE651/Project/submit/gearmax-data/test_data/mini_data


In [17]:
car_option.loc[car_option['car_id'] == 4435]

,option_id,car_id
2123,3,4435
4448,4,4435
7095,6,4435
8553,9,4435
10796,17,4435
21545,29,4435
22826,31,4435
24599,35,4435
25307,37,4435
29701,41,4435


In [18]:
# print (major_option[0])
car_option.shape

(62431, 2)

In [19]:
cars['major_options'][4435]

"['Sunroof/Moonroof', 'Navigation System', 'Adaptive Cruise Control', 'Third Row Seating', 'Adaptive Suspension', 'Blind Spot Monitoring', 'Parking Sensors', 'Heated Seats', '302A Luxury Equipment Group', 'Tow Package', 'Android Auto', 'CarPlay', 'Luxury Package', 'Backup Camera', 'Convenience Package']"

In [30]:
df = cars.loc[cars['major_options'].str.contains('Navigation System', na=False), 'major_options']
# df.shape
# df[9362]

In [31]:
Car.head(2)

,vin,is_depreciated,seller_id,zip,city,country,price,year,make_name,model_name,...,fuel_tank_volume,fuel_type,city_fuel_economy,highway_fuel_economy,is_new,listed_date,main_picture_url,owner_count,seller_rating,trim_name
0,ZACNJABB5KPJ92081,False,370599.0,960,Bayamon,USA,23141.0,2019,Jeep,Renegade,...,12.7,Gasoline,NaN,NaN,True,2019-04-06,https://static.cargurus.com/images/forsale/202...,NaN,2.8,Latitude FWD
1,SALCJ2FX1LH858117,False,389227.0,922,San Juan,USA,46500.0,2020,Land Rover,Discovery Sport,...,17.7,Gasoline,NaN,NaN,True,2020-02-15,https://static.cargurus.com/images/forsale/202...,NaN,3.0,S AWD
